In [1]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import pandas as pd

In [2]:
# from tensorflow.python.client import device_lib
data = pd.read_csv("database_some.csv")
# device_lib.list_local_devices()

In [3]:
data.head()

,Title,Text,Time,Url,Value
0,VK может завершить проект производства умных к...,© Reuters. VK может завершить проект производс...,"13.11.2024, 10:38",https://ru.investing.com/news/general/article-...,-0.116815
1,VK может завершить проект производства умных к...,© Reuters. VK может завершить проект производс...,"13.11.2024, 10:38",https://ru.investing.com/news/general/article-...,-0.085109
2,Золото на одномесячном минимуме из-за крепкого...,© Reuters.\nInvesting.com — Золото торгуется н...,"12.11.2024, 09:38",https://ru.investing.com/news/commodities-news...,-0.000494
3,"Рынок акций РФ закрылся ростом, Индекс МосБирж...","© Reuters. Рынок акций РФ закрылся ростом, Инд...","31.10.2024, 00:20",https://ru.investing.com/news/stock-market-new...,0.030545
4,"Рынок акций РФ закрылся ростом, Индекс МосБирж...","Рынок акций РФ закрылся ростом, Индекс МосБирж...","30.10.2024, 00:20",https://ru.investing.com/news/stock-market-new...,-0.030496


In [4]:
# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Tokenize text
def tokenize_data(texts, labels, max_len=128):
    inputs = tokenizer(texts.tolist(), max_length=max_len, truncation=True, padding=True, return_tensors='tf')
    return inputs, tf.convert_to_tensor(labels)

In [6]:
# Prepare data
texts = data['Text']
labels = data['Value']
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)


In [7]:
print(labels)

0      -0.116815
1      -0.085109
2      -0.000494
3       0.030545
4      -0.030496
          ...   
4666    0.966649
4667    0.966649
4668    0.966649
4669    0.966649
4670    0.966649
Name: Value, Length: 4671, dtype: float64


In [8]:
train_inputs, train_labels = tokenize_data(X_train, y_train)
test_inputs, test_labels = tokenize_data(X_test, y_test)

In [9]:
# print("train_labels:", train_labels)
print("train_labels shape:", train_labels.shape)
print("train_labels dtype:", train_labels.dtype)

# print("test_labels:", test_labels)
print("test_labels shape:", test_labels.shape)
print("test_labels dtype:", test_labels.dtype)

train_labels shape: (3736,)
train_labels dtype: <dtype: 'float64'>
test_labels shape: (935,)
test_labels dtype: <dtype: 'float64'>


In [10]:
print("train_inputs shape:", train_inputs["input_ids"].shape)
print("train_labels shape:", train_labels.shape)

train_inputs shape: (3736, 128)
train_labels shape: (3736,)


In [11]:
print("train_inputs shape:", test_inputs["input_ids"].shape)
print("train_labels shape:", test_labels.shape)

train_inputs shape: (935, 128)
train_labels shape: (935,)


In [12]:
# Compile and train
model.compile(optimizer="adam", loss=model.compute_loss, metrics=['accuracy'])



In [13]:
model.fit(
    x={'input_ids': train_inputs['input_ids'], 'attention_mask': train_inputs['attention_mask']},
    y=train_labels,
    validation_data=({"input_ids": test_inputs["input_ids"], "attention_mask": test_inputs["attention_mask"]}, test_labels),
    epochs=3,
    batch_size=16
)

Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


AttributeError: in user code:

    File "/home/almaz/anaconda3/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/home/almaz/anaconda3/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1370, in run_step  *
        outputs = model.train_step(data)
    File "/home/almaz/anaconda3/lib/python3.12/site-packages/transformers/modeling_tf_utils.py", line 1706, in train_step  *
        loss = self.compiled_loss(y, y_pred, sample_weight, regularization_losses=self.losses)
    File "/home/almaz/anaconda3/lib/python3.12/site-packages/tf_keras/src/engine/compile_utils.py", line 275, in __call__  *
        y_t, y_p, sw = match_dtype_and_rank(y_t, y_p, sw)
    File "/home/almaz/anaconda3/lib/python3.12/site-packages/tf_keras/src/losses.py", line 143, in __call__  *
        losses = call_fn(y_true, y_pred)
    File "/home/almaz/anaconda3/lib/python3.12/site-packages/tf_keras/src/losses